# Clustering

## Setup

In [ ]:
%conda install -c conda-forge numpy pandas matplotlib seaborn scipy

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from scipy.spatial.distance import cdist
from scipy.stats import multivariate_normal
import time

In [ ]:
data_pca = np.load('data/features_pca.npz')
features_pca = data_pca['features_pca']
movieIds_pca = data_pca['movieIds_pca']

data_svd = np.load('data/features_svd.npz')
features_svd = data_svd['features_svd']
movieIds_svd = data_svd['movieIds_svd']

data_lda = np.load('data/features_lda.npz')
features_lda = data_lda['features_lda']
movieIds_lda = data_lda['movieIds_lda']

movies = pd.read_csv('data/train_complete.csv')
movies = pd.concat([movies, pd.read_csv('data/test_complete.csv')], ignore_index=True)

## Selected Algorithm

1. K-means (Partitioning method)
2. Gaussian Mixture Model - GMM (Distriburion-based method)